# Cluster Forecasting

In [48]:
import pandas as pd
import sketch
import ast

In [2]:
clients_per_clusters_path = 'clean_data/clients_per_clusters.csv'
transactions_path = 'train_data_npo/npo_trnsctns.csv'
contributors_path = 'train_data_npo/npo_cntrbtrs.csv'

clients_features = {
    "clnt_id" : "ID клиента",
    "slctn_nmbr" : "Номер выборки",
    "gndr" : "Пол клиента: м: 0, ж: 1",
    "age" : "Возраст клиента, лет",
    "brth_yr" : "Год рождения клиента",
    "pstl_code" : "Почтовый индекс",
    "city" : "Живет клиент в городе (1) или нет (0)"
}

contributors_features = {
    "npo_accnt_id" : "ID счета клиента",
    "client_id" : "ID клиента",
    "slctn_nmbr" : "Номер выборки",
    "accnt_pnsn_schm" : "Условный код пенсионной схемы счета клиента",
    "npo_accnt_status" : "Статус счета клиента: Закрыт: 0, Открыт: 1",
    "npo_accnt_status_date" : "Дата статуса счета клиента",
    "npo_blnc" : "Баланс счета клиента, руб.",
    "npo_pmnts_sum" : "Сумма взносов клиента, руб.",
    "npo_pmnts_nmbr" : "Число взносов клиента",
    "npo_frst_pmnt_date" : "Дата первого взноса клиента",
    "npo_lst_pmnt_date" : "Дата последнего взноса клиента",
    "npo_ttl_incm" : "Сумма дохода, начисленного на счет клиента, руб."
}

transactions_features = {
    "npo_accnt_id" : "ID клиента",
    "slctn_nmbr" : "Номер выборки",
    "npo_sum" : "Размер операции по счету клиента, руб.",
    "npo_operation_date" : "Дата взноса клиента",
    "npo_operation_group" : "Тип операции по счету клиента: Поступление взносов: 0, Начисление дохода: 1",
}

In [5]:
clients_per_clusters = pd.read_csv(clients_per_clusters_path, index_col=0)
clients_per_clusters.head(3)

,cluster,clnt_id
0,0,"['0x896FDAA89D08B44698884BC6876C3455', '0x7F16..."
1,1,"['0x88C8CB57D2D6B14393894C0CBB8A9A4A', '0x1441..."
2,2,"['0xE161325D909F9848979ABFE4137216CE', '0xE459..."


In [6]:
transactions = pd.read_csv(transactions_path)
transactions.head(3)

,npo_accnt_id,npo_sum,slctn_nmbr,npo_operation_date,npo_operation_group
0,0x05C7DF8BA2611640BE946E29CF20C6D2,1626.01,0,2011-11-11 00:00:00,0
1,0x05C7DF8BA2611640BE946E29CF20C6D2,8394.05,0,2010-08-30 00:00:00,1
2,0x05C7DF8BA2611640BE946E29CF20C6D2,1626.01,0,2007-03-12 00:00:00,0


In [60]:
client_payments = transactions.loc[transactions['npo_operation_group'] == 0, 
    ['npo_accnt_id', 'npo_sum', 'slctn_nmbr', 'npo_operation_date']]
# Rename column
client_payments.rename(columns={'npo_accnt_id':'clnt_id'}, inplace=True)
client_payments['clnt_id'] = client_payments['clnt_id'].astype(str)
print(client_payments.info())
client_payments.head(3)

<class 'pandas.core.frame.DataFrame'>
Index: 9711822 entries, 0 to 12325095
Data columns (total 4 columns):
 #   Column              Dtype  
---  ------              -----  
 0   clnt_id             object 
 1   npo_sum             float64
 2   slctn_nmbr          int64  
 3   npo_operation_date  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 370.5+ MB
None


,clnt_id,npo_sum,slctn_nmbr,npo_operation_date
0,0x05C7DF8BA2611640BE946E29CF20C6D2,1626.01,0,2011-11-11 00:00:00
2,0x05C7DF8BA2611640BE946E29CF20C6D2,1626.01,0,2007-03-12 00:00:00
4,0x05C7DF8BA2611640BE946E29CF20C6D2,1626.01,0,2016-11-05 00:00:00


In [62]:
client_payments.sketch.howto("Group by npo_operation_date ")

,clnt_id,npo_sum,slctn_nmbr,npo_operation_date
0,0x05C7DF8BA2611640BE946E29CF20C6D2,1626.01,0,2011-11-11 00:00:00
2,0x05C7DF8BA2611640BE946E29CF20C6D2,1626.01,0,2007-03-12 00:00:00
4,0x05C7DF8BA2611640BE946E29CF20C6D2,1626.01,0,2016-11-05 00:00:00


1 квартал 2005 год